<a href="https://colab.research.google.com/github/plusheart/NTHU_2025_GenAI/blob/main/Dict/w7_%E5%85%A9LLM%E8%AC%9B%E7%9B%B8%E8%81%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.0 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import gradio as gr
from openai import OpenAI
import time

In [ ]:
import os
from google.colab import userdata

In [ ]:
#【使用 Groq】

api_key = userdata.get('Groq')
base_url="https://api.groq.com/openai/v1"

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

In [ ]:
os.environ['OPENAI_API_KEY']=api_key

In [ ]:
# 系統設定
main_model = "llama3-70b-8192"
main_name = "阿呆"
sidekick_name = "阿瓜"

# 角色設定

In [ ]:
# system prompt: 設定黑色幽默風格的相聲角色
main_sys = f"""
你叫{main_name}，是個講黑色幽默相聲的主角。
風格要諷刺、幽默中帶有現實的殘酷。
每次發言約25~40字，不要太長，保有節奏感。
"""

sidekick_sys = f"""
你叫{sidekick_name}，是{main_name}的捧哏搭檔，會針對他剛才說的吐槽、接話、表示誇張或反應，風格幽默但白目。
每次講話不要超過30字，要讓對話有節奏與趣味。
"""

# 兩模型對話之相聲

*   輪流回應，逐步更新
*   使用者決定主題跟話題總次數



In [ ]:
def stream_crosstalk(topic, rounds):
    history = []

    # 🔹 第一句：阿呆開場
    main_prompt = [
        {"role": "system", "content": main_sys},
        {"role": "user", "content": f"我們來說一段關於「{topic}」的黑色幽默相聲，開始第一句。"}
    ]
    main_reply = client.chat.completions.create(
        model=main_model,
        messages=main_prompt
    ).choices[0].message.content.strip()
    history.append((main_name, main_reply))
    yield [(main_reply, None)]

    current_reply = main_reply
    chatbot_ui = [(main_reply, None)]

    # 輪流回合
    for i in range(rounds - 1):
        if i % 2 == 0:
            # 捧哏阿瓜回應
            side_prompt = [
                {"role": "system", "content": sidekick_sys},
                {"role": "user", "content": f"{main_name}剛剛說：『{current_reply}』，你要怎麼接？"}
            ]
            side_reply = client.chat.completions.create(
                model=main_model,
                messages=side_prompt
            ).choices[0].message.content.strip()
            history.append((sidekick_name, side_reply))
            current_reply = side_reply
            chatbot_ui[-1] = (chatbot_ui[-1][0], side_reply)
        else:
            # 主角阿呆回應
            main_prompt = [
                {"role": "system", "content": main_sys},
                {"role": "user", "content": f"{sidekick_name}剛剛說：『{current_reply}』，你怎麼回應？"}
            ]
            main_reply = client.chat.completions.create(
                model=main_model,
                messages=main_prompt
            ).choices[0].message.content.strip()
            history.append((main_name, main_reply))
            current_reply = main_reply
            chatbot_ui.append((main_reply, None))

        yield chatbot_ui
        time.sleep(0.3)  # 時間延遲使節奏更自然

# Gradio 做介面

In [ ]:
# 新增對話頭像
avatar_adai = "/content/drive/MyDrive/Colab Notebooks/GenAI/adai.jpg"
avatar_agua = "/content/drive/MyDrive/Colab Notebooks/GenAI/agua.jpg"

In [ ]:
# Gradio UI 建立
with gr.Blocks(title="黑色幽默相聲 🎭 阿呆與阿瓜") as demo:
    gr.Markdown("## 🎤 黑色幽默相聲機器人<br>阿呆（左） & 阿瓜（右）")

    topic_input = gr.Textbox(label="主題", placeholder="輸入主題（如：裁員、失戀、人生無常）")
    round_input = gr.Slider(minimum=2, maximum=12, step=2, value=6, label="對話總次數（偶數）")
    btn = gr.Button("🎬 開始相聲")

    # 加上頭像
    chatbot = gr.Chatbot(
        label="阿呆與阿瓜",
        height=400,
        avatar_images=(avatar_adai, avatar_agua)
    )

    btn.click(fn=stream_crosstalk, inputs=[topic_input, round_input], outputs=chatbot)

demo.launch(share=True)

<ipython-input-61-4232e7b9023b>:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9376f96910bb40f4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
